In [1]:
import sage.logic.propcalc as pc
import itertools as it
import numpy.random as rn
from sage.plot.histogram import Histogram
import numpy as np

In [2]:
# Define the Boolean Rules
Rules = """E2F1 *= (~pRB & E2F1) | (~pRB & Myc)
CyclinD1 *= (~p21 & Myc) | (~pRB & Myc & E2F1) | (~p21 & E2F1) | (~pRB & E2F1 & CyclinD1)
p21 *= ~CyclinE
Myc *= ~pRB & E2F1
CyclinE *= ~pRB & ~p27Kip1 & E2F1
pRB *= (p27Kip1 & ~CyclinD1) | (~CyclinE & ~CyclinD1)
p27Kip1 *= ~CyclinE & ~CyclinD1"""

In [3]:
# Parse the Boolean Rules
rdict = {}
for line in Rules.split('\n'):
    lr = line.split('*=')
    rdict[lr[0].strip()] = pc.formula(lr[1])
    
for r in rdict: print r, "*=", rdict[r]

E2F1 *= (~pRB&E2F1)|(~pRB&Myc)
CyclinE *= ~pRB&~p27Kip1&E2F1
pRB *= (p27Kip1&~CyclinD1)|(~CyclinE&~CyclinD1)
p27Kip1 *= ~CyclinE&~CyclinD1
p21 *= ~CyclinE
CyclinD1 *= (~p21&Myc)|(~pRB&Myc&E2F1)|(~p21&E2F1)|(~pRB&E2F1&CyclinD1)
Myc *= ~pRB&E2F1


In [4]:
# Convert the rules to Hill kinetics rate functions
x={r:var('x_'+r) for r in rdict}
RF = {}
for r in rdict:
    k = {}
    n = {}
    H = {}
    
    RF[r] = -x[r]
    
    varnames = str(rdict[r].truthtable(end=0)).split()[0:-1]
    
    for ncomb in range(len(varnames)+1):
        for comb in it.combinations(varnames,ncomb):
            st = {v: v in list(comb) for v in varnames}
            hprod = 0
            if rdict[r].evaluate(st):           
                hprod = 1
                for v in varnames:
                    rv = r + '_' + v
                    k[rv] = var('k_' + rv)
                    n[rv] = var('n_' + rv)
                    
                    if st[v]:
                        hprod *= x[v]^n[rv] / (k[rv]^n[rv] + x[v]^n[rv])
                    else:
                        hprod *= k[rv]^n[rv] / (k[rv]^n[rv] + x[v]^n[rv])
            RF[r] += hprod
    RF[r] = ((RF[r]+x[r]).numerator()/(RF[r]+x[r]).denominator()) - x[r]

In [5]:
# Function to help obtain the "worst case system" of Rozum & Albert 2018 (PLOS Comp. Bio)
# Given v and lists vmin and vmax, returns the rate function for v with the variables in vmin set to zero
# and variables in vmax set to infinity
def wcRF(v, vmin, vmax):
    vmin = set([x[vmx] for vmx in vmin])
    vmax = set([x[vmx] for vmx in vmax])
    mn1 = 0
    md1 = 0
    
    if (RF[v]+x[v]).numerator().operator() == sage.symbolic.operators.mul_vararg:
        numerTerms = [(RF[v]+x[v]).numerator()]
    else:
        numerTerms = (RF[v]+x[v]).numerator().operands()
    
    for t in numerTerms:
        if vmin.isdisjoint(set(t.args())) and vmax.issubset(set(t.args())):
            for vmx in vmax:
                t = t.subs(vmx == 1)
            mn1 += t
    for t in (RF[v]+x[v]).denominator().expand().operands():
        for vmx in vmin:
            t = t.subs(vmx == 0)
        if vmax.issubset(set(t.args())):
            for vmx in vmax:
                t = t.subs(vmx == 1)
            md1 += t
    #for t in (RF[v]+x[v]).denominator().operands():
    #    if vmax.isdisjoint(set(t.args())):
    #        md1 *= t
    return simplify(mn1/md1 - x[v])

In [7]:
# The worst case system for this motif
wcCycD=wcRF("CyclinD1",[],["Myc","E2F1"])
wcpRB=wcRF("pRB",["p27Kip1"],[])
wcCycE=wcRF("CyclinE",["p27Kip1"],["E2F1"])
wcp21=RF["p21"]

In [8]:
show(wcCycD)
show(wcpRB)
show(wcCycE)
show(wcp21)

-x_CyclinD1 + (k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*k_CyclinD1_p21^n_CyclinD1_p21*k_CyclinD1_pRB^n_CyclinD1_pRB + k_CyclinD1_p21^n_CyclinD1_p21*k_CyclinD1_pRB^n_CyclinD1_pRB*x_CyclinD1^n_CyclinD1_CyclinD1 + k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*k_CyclinD1_pRB^n_CyclinD1_pRB*x_p21^n_CyclinD1_p21 + k_CyclinD1_pRB^n_CyclinD1_pRB*x_CyclinD1^n_CyclinD1_CyclinD1*x_p21^n_CyclinD1_p21 + k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*k_CyclinD1_p21^n_CyclinD1_p21*x_pRB^n_CyclinD1_pRB + k_CyclinD1_p21^n_CyclinD1_p21*x_CyclinD1^n_CyclinD1_CyclinD1*x_pRB^n_CyclinD1_pRB)/(k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*k_CyclinD1_p21^n_CyclinD1_p21*k_CyclinD1_pRB^n_CyclinD1_pRB + k_CyclinD1_p21^n_CyclinD1_p21*k_CyclinD1_pRB^n_CyclinD1_pRB*x_CyclinD1^n_CyclinD1_CyclinD1 + k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*k_CyclinD1_pRB^n_CyclinD1_pRB*x_p21^n_CyclinD1_p21 + k_CyclinD1_pRB^n_CyclinD1_pRB*x_CyclinD1^n_CyclinD1_CyclinD1*x_p21^n_CyclinD1_p21 + k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*k_CyclinD1_p21^n_CyclinD1_p21*x_pRB^n_CyclinD1_pRB + k_CyclinD1_p21^n_CyclinD1_p21*x_CyclinD1^n_CyclinD1_CyclinD1*x_pRB^n_CyclinD1_pRB + k_CyclinD1_CyclinD1^n_CyclinD1_CyclinD1*x_p21^n_CyclinD1_p21*x_pRB^n_CyclinD1_pRB + x_CyclinD1^n_CyclinD1_CyclinD1*x_p21^n_CyclinD1_p21*x_pRB^n_CyclinD1_pRB)

k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1/(k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1*x_CyclinD1^n_pRB_CyclinD1 + k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_p27Kip1^n_pRB_p27Kip1*x_CyclinE^n_pRB_CyclinE + k_pRB_p27Kip1^n_pRB_p27Kip1*x_CyclinD1^n_pRB_CyclinD1*x_CyclinE^n_pRB_CyclinE) - x_pRB

k_CyclinE_p27Kip1^n_CyclinE_p27Kip1*k_CyclinE_pRB^n_CyclinE_pRB/(k_CyclinE_p27Kip1^n_CyclinE_p27Kip1*k_CyclinE_pRB^n_CyclinE_pRB + k_CyclinE_p27Kip1^n_CyclinE_p27Kip1*x_pRB^n_CyclinE_pRB) - x_CyclinE

-x_p21 + k_p21_CyclinE^n_p21_CyclinE/(k_p21_CyclinE^n_p21_CyclinE + x_CyclinE^n_p21_CyclinE)

In [9]:
# Simplify at steady state
wcCycD=(wcCycD+x["CyclinD1"]).simplify_rational()-x["CyclinD1"]
wcpRB=(wcpRB+x["pRB"]).simplify_rational()-x["pRB"]
wcCycE=(wcCycE+x["CyclinE"]).simplify_rational()-x["CyclinE"]
wcp21=(wcp21+x["p21"]).simplify_rational()-x["p21"]
show(wcCycD)
show(wcpRB)
show(wcCycE)
show(wcp21)

-x_CyclinD1 + (k_CyclinD1_p21^n_CyclinD1_p21*k_CyclinD1_pRB^n_CyclinD1_pRB + k_CyclinD1_pRB^n_CyclinD1_pRB*x_p21^n_CyclinD1_p21 + k_CyclinD1_p21^n_CyclinD1_p21*x_pRB^n_CyclinD1_pRB)/(k_CyclinD1_p21^n_CyclinD1_p21*k_CyclinD1_pRB^n_CyclinD1_pRB + k_CyclinD1_pRB^n_CyclinD1_pRB*x_p21^n_CyclinD1_p21 + (k_CyclinD1_p21^n_CyclinD1_p21 + x_p21^n_CyclinD1_p21)*x_pRB^n_CyclinD1_pRB)

k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE/(k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE + k_pRB_CyclinE^n_pRB_CyclinE*x_CyclinD1^n_pRB_CyclinD1 + (k_pRB_CyclinD1^n_pRB_CyclinD1 + x_CyclinD1^n_pRB_CyclinD1)*x_CyclinE^n_pRB_CyclinE) - x_pRB

-x_CyclinE + k_CyclinE_pRB^n_CyclinE_pRB/(k_CyclinE_pRB^n_CyclinE_pRB + x_pRB^n_CyclinE_pRB)

-x_p21 + k_p21_CyclinE^n_p21_CyclinE/(k_p21_CyclinE^n_p21_CyclinE + x_CyclinE^n_p21_CyclinE)

In [10]:
# Define a threshold variable; we will study pRB > TE
var('TE')

TE

In [11]:
# A functional motif will occur if pRB-TE=pPRB=0 has a solution
TCYCE=wcCycE.subs(x["pRB"]==TE).solve(x["CyclinE"])[0].rhs()
TP21=wcp21.subs(x["CyclinE"]==TCYCE).solve(x["p21"])[0].rhs()
TCYCD=wcCycD.subs(x["p21"]==TP21,x["pRB"]==TE).solve(x["CyclinD1"])[0].rhs()
pPRB=wcpRB.subs(x["CyclinE"]==TCYCE,x["CyclinD1"]==TCYCD)

In [12]:
# Function to search for roots of f on an interval [a,b] with tolerance eps
def find_all_roots(f, a, b, eps=0.001):
    roots = []
    intervals_to_check = [(a,b)]
    while intervals_to_check:
        start, end = intervals_to_check.pop()
        try:
            root = find_root(f, start, end)
        except RuntimeError:
            continue
        if root in roots:
            continue
        if abs(f(root)) < 1:
            roots.append(root)
        intervals_to_check.extend([(start, root-eps), (root+eps, end)])
    roots.sort()
    return roots

# Find the midpoint of the line segment connecting (x1,y1) to (x2,y2)
def linInterpRoot(x1,x2,y1,y2):
    m = (y2-y1)/(x2-x1)
    b = y1-x1*m
    x0 = -b/m
    return x0
    
# Class to store parameter sets and test them for functionality    
class SMRoot:
    Nscan = 20
    tscan = [i/Nscan for i in range(1,Nscan+1)]
    lowroot = 0.499
    highroot = 1
    eps = 0.001

    def __init__(self,paramDict):
        self.paramDict=paramDict
        self.maxRootDict = {}
        self.tested = False
        
    def buildMaxRoot(self):
        r = find_all_roots(pPRB.subs(self.paramDict).subs(x["pRB"]==TE),0.5,1)
        if len(r) > 0:
            self.maxRootDict = {"pRB":r[-1],
                            "CyclinE":TCYCE.subs(self.paramDict).subs(TE==r[-1]),
                            "p21":TP21.subs(self.paramDict).subs(TE==r[-1]),
                            "CyclinD1":TCYCD.subs(self.paramDict).subs(TE==r[-1])}
       
        self.tested = True

In [13]:
# Read unspecified parameters and build a function to randomly generate values for them
kargs = []
nargs = []
for v in pPRB.args() + wcCycD.args():
    if str(v)[0]=="k":
        kargs.append(v)
    if str(v)[0]=="n":
        nargs.append(v)

kargs = list(set(kargs))
nargs = list(set(nargs))
nkargs = nargs + kargs
   

def genRandomParams(seed = None, scale = "Log", nValBounds = (1,8),kValBounds = (1/16,15/16)):
    rn.seed(seed)
    
    # sample low values exponentially more often than high values (uniform in log-space); otherwise, uniform in linear-space
    if scale == "Log":
        nvals = [-ln(xx) for xx in rn.uniform(exp(-nValBounds[0]),exp(-nValBounds[1]),len(nargs))]
        kvals = [-ln(xx) for xx in rn.uniform(exp(-kValBounds[0]),exp(-kValBounds[1]),len(kargs))]
    else:
        nvals = [xx for xx in rn.uniform(nValBounds[0],nValBounds[1],len(nargs))]
        kvals = [xx for xx in rn.uniform(kValBounds[0],kValBounds[1],len(kargs))]
        
    nkvals = nvals + kvals
    return {nkargs[i]:nkvals[i] for i in range(len(nkvals))}

In [16]:
# Test functionality for NSamples parameter sets for this candidate motif
NSamples = 10000

SM = []
NSM = []
strongSM = []
for i in range(NSamples):
    testDict=genRandomParams(seed=i,scale="Lin")
    testRoot = SMRoot(testDict)
    testRoot.buildMaxRoot()
    if len(testRoot.maxRootDict.keys()) > 0:
        SM.append(testRoot)
        r = testRoot.maxRootDict
        if r["pRB"] > 0.5 and r["CyclinD1"] < 0.5 and r["CyclinE"] < 0.5 and r["p21"] > 0.5:
            strongSM.append(testRoot)
    else:
        NSM.append(testRoot)


/usr/lib/sagemath/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:37: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.


In [19]:
# Save the output
save(SM,'SM3')
save(strongSM,'strongSM3')
save(NSM,'NSM3')

In [14]:
# Load output if it already exists
NSM = load('./NSM3.sobj')
SM = load('./SM3.sobj')
strongSM = load('./strongSM3.sobj')

In [18]:
# Some basic statistics
print(float(len(SM))/(len(NSM) + len(SM)))
print(float(len(strongSM))/(len(SM)))
print(float(len(strongSM))/(len(NSM) + len(SM)))

0.3685
0.959837177748
0.3537
